# CT-LTI: Figure 5
Evaluation of control similarity between NODEC and OC via correlation of control signals per node and energy plots.


Furthermore, please make sure that the required data folder is available at the paths used by the script.
You may generate the required data by running the python script
```nodec_experiments/ct_lti/gen_parameters.py```.

Please also make sure that a trainingproceedure has produced results in the corresponding paths used below.
Running ```nodec_experiments/ct_lti/single_sample/train.ipynb``` with default paths is expected to generate at the requiered location.

As neural network intialization is stochastic, please make sure that appropriate seeds are used or expect some variance to paper results.

## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.sys.path.append('../../../')

import torch
from torchdiffeq import odeint

import numpy as np
import pandas as pd
import networkx as nx

import plotly
from copy import deepcopy
import scipy

from plotly import graph_objects as go
import plotly.express as px

from tqdm.auto import tqdm
from nnc.helpers.plot_helper import square_lattice_heatmap, trendplot
from nnc.helpers.torch_utils.file_helpers import read_tensor_from_collection, \
    save_tensor_to_collection

In [ ]:
from nnc.helpers.plot_helper import ColorRegistry, base_layout


## Loading parameters and data

In [ ]:
results_data_folder = '../../../../results/ct_lti/single_sample/'
experiment_data_folder = '../../../../data/parameters/ct_lti/'
graph='lattice'

n_interactions = ['50', '500', '5000']
collection_file = 'epochs.zip'

evaluation_files =  dict(
oc_50 = results_data_folder + 'oc_sample_ninter_50/',
oc_500 = results_data_folder + 'oc_sample_ninter_500/',
oc_5000 = results_data_folder + 'oc_sample_ninter_5000/',

nodec_50 = results_data_folder + 'eval_nn_sample_ninter_50/',
nodec_500 = results_data_folder + 'eval_nn_sample_ninter_500/',
nodec_5000 = results_data_folder + 'eval_nn_sample_ninter_5000/',
)

all_files  = dict(
    train_file = results_data_folder + 'nn_sample_train/',
)
all_files.update(evaluation_files)

In [ ]:
graph='lattice'
graph_folder = experiment_data_folder+graph+'/'
device='cpu'
target_states = torch.load(graph_folder+'target_states.pt').to(device)
initial_states = torch.load(experiment_data_folder+'init_states.pt').to(device)

current_sample_id = 24

x0 = initial_states[current_sample_id].unsqueeze(0)
xstar = target_states[current_sample_id].unsqueeze(0)
T = 0.5


## Fig: 5a
Correlation scatter plot between control signals of NODEC and OC with fitter OLS.

In [ ]:
oc_controls = read_tensor_from_collection(all_files['oc_500'] + 'epochs.zip', 'all_controls/ep_0.pt')
nnc_controls = read_tensor_from_collection(all_files['nodec_500']+ 'epochs.zip', 'all_controls/ep_0.pt')
fig_ols = trendplot(x1=oc_controls[0, :, :].flatten().cpu().numpy(), x2=nnc_controls[0, :, :].flatten().cpu().numpy(),
         ax1='OC', ax2='NODEC', render_mode='webgl'
         )

fig_ols.data[0].showlegend = False
#fig_ols.data[0].name = 'Control Points'
fig_ols.data[1].name = 'OLS '
fig_ols.data[1].showlegend = True
fig_ols.update_layout(            height = fig_ols.layout.height-fig_ols.layout.margin.t+10,
                                  margin = dict(t=0),
                                  legend=dict(
                                        orientation="v",
                                  x=0,
                                  y=1.1,                                
                                  bgcolor="rgba(0,0,0,0)",
                                  bordercolor="Black",
                                  borderwidth=0
                                )

                                  )
fig_ols.layout.annotations[0].x = 0.23
fig_ols.layout.annotations[0].y = 0


fig_ols.update_layout(width=400, height=300)
fig_ols



## Fig 5b
Energy comparison between OC and NODEC trajectories over a single sample.

In [ ]:
oc_500_energies = read_tensor_from_collection(evaluation_files['oc_500'] + 'epochs.zip', 'all_energies/ep_0.pt')
nn_500_energies = read_tensor_from_collection(evaluation_files['nodec_500'] + 'epochs.zip', 'all_energies/ep_0.pt')
timesteps = torch.linspace(0, T, 500).numpy()


nodec_500_energies_time = px.line(x=timesteps, y=nn_500_energies.flatten().cpu().numpy()).data[0]
nodec_500_energies_time.name = 'NODEC'
nodec_500_energies_time.line.color = ColorRegistry.nodec
nodec_500_energies_time.showlegend = True 
nodec_500_energies_time.line.dash = 'dot'
oc_500_energies_time = px.line(x=timesteps, y=oc_500_energies.flatten().cpu().numpy()).data[0]
oc_500_energies_time.name = 'OC'
oc_500_energies_time.line.color = ColorRegistry.oc
oc_500_energies_time.showlegend = True 
oc_500_energies_time.line.dash = 'dot'

energy_figure = go.Figure([nodec_500_energies_time, oc_500_energies_time])
energy_figure.update_layout(base_layout)

energy_figure.update_layout(width=145, height=137, margin = dict(t=0,b=0,l=0,r=0), 
                                  legend=dict(
                                        orientation="h",
                                  font = dict(size=8),
                                  x=0,
                                  y=1.1,                                
                                  bgcolor="rgba(0,0,0,0)",
                                  bordercolor="Black",
                                  borderwidth=0
                                  )
                                )

energy_figure.layout.yaxis.title = 'Total Energy'
energy_figure.layout.xaxis.title = 'Time'
energy_figure.update_layout(width=400, height=300)

energy_figure